En esta fase dejamos el dataset listo para entrenar los modelos del proyecto **IA-CANCER-LOGISTIC-RF**. Con base en lo observado en la Fase 1, aplicamos los pasos necesarios para limpiar, transformar y organizar los datos, evitando problemas como *data leakage* y asegurando que cada modelo reciba la información en el formato adecuado.

El objetivo principal es construir las matrices de entrenamiento y prueba que usarán la **Regresión Logística** y el **Random Forest**, teniendo en cuenta que la primera requiere datos escalados, mientras que el segundo puede trabajar con las características originales.

Aquí realizaremos las tareas esenciales del preprocesamiento: eliminar columnas irrelevantes, codificar la variable objetivo, separar predictores y etiquetas, dividir el dataset, escalar únicamente lo necesario y guardar el escalador para su uso posterior. Con estos pasos completados, los datos quedarán preparados para iniciar la Fase 3, donde entrenaremos los modelos supervisados.

In [1]:
import pandas as pd

# Ruta al archivo (ajústala si tu estructura es distinta)
ruta = "../data/Cancer_Data.csv"   # si el notebook está en notebooks/
# ruta = "data/Cancer_Data.csv"    # si el notebook está en la raíz del proyecto

# Cargar el dataset
df = pd.read_csv(ruta)

A continuación, se eliminarán las columnas que no aportan información relevante al modelo. Este tipo de depuración es parte esencial del proceso de construcción de modelos, ya que permite quedarse únicamente con las variables útiles para la predicción. Mantener columnas como id podría llevar a que el modelo aprenda patrones irrelevantes o simples ruidos del dataset, por lo que retirarlas asegura un entrenamiento más limpio y coherente con los objetivos del análisis.

In [2]:
# Eliminar columnas que no aportan valor al modelo
df = df.drop(columns=["id", "Unnamed: 32"], errors="ignore")

# Verificar que ya no existan
df.head()
df.shape

(569, 31)

En las siguientes líneas de código se realizará la codificación de la variable objetivo para convertirla a formato numérico. Este paso es necesario porque modelos supervisados como Regresión Logística y Random Forest requieren que la salida sea numérica y no categórica con letras. Además, este tipo de codificación (0/1) es el estándar en problemas de clasificación binaria y coincide con los ejemplos trabajados en el material del curso.

In [3]:
# Codificación binaria de la variable objetivo
df["diagnosis"] = df["diagnosis"].map({"B": 0, "M": 1})

# Verificar que la transformación quedó correcta
df["diagnosis"].value_counts()


diagnosis
0    357
1    212
Name: count, dtype: int64

A continuación, se separarán las variables predictoras y la variable objetivo para preparar el dataset en el formato adecuado antes de realizar la división en entrenamiento y prueba. Esta estructuración es la que se utiliza de manera estándar en los flujos de trabajo de Python y coincide con los ejemplos vistos en clase: por un lado, X contendrá únicamente las características numéricas que se usarán para hacer la predicción, mientras que y almacenará los valores codificados (0/1) correspondientes al diagnóstico.

In [4]:
# Separar X (predictores) e y (variable objetivo)
X = df.drop(columns=["diagnosis"])
y = df["diagnosis"]

# Revisar dimensiones
X.shape, y.shape

((569, 30), (569,))

A continuación, se realiza la división del dataset en entrenamiento y prueba. Es importante hacerlo antes de aplicar cualquier tipo de escalamiento para evitar data leakage, tal como se recomienda en los flujos de trabajo correctos de entrenamiento. De esta manera, las transformaciones se ajustan únicamente con los datos de entrenamiento y luego se aplican al conjunto de prueba. Además, se utiliza el parámetro stratify=y para mantener el mismo balance de clases en ambos subconjuntos, asegurando una evaluación más justa del modelo.

In [5]:
from sklearn.model_selection import train_test_split

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20, 
    random_state=42, 
    stratify=y   # mantiene la proporción de clases
)

# Verificar tamaños
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((455, 30), (114, 30), (455,), (114,))

La salida mostrada corresponde a las dimensiones generadas por train_test_split. El conjunto X_train contiene 455 observaciones y 30 variables predictoras, lo que representa aproximadamente el 80% del total y será el que utilicemos para entrenar los modelos. Por su parte, X_test incluye 114 observaciones con las mismas 30 características y corresponde al 20% restante, reservado exclusivamente para evaluar el desempeño del modelo. De forma paralela, y_train y y_test almacenan las etiquetas asociadas a cada conjunto, con 455 y 114 valores respectivamente, codificados como 0 (benigno) y 1 (maligno). Esta distribución confirma que la partición se realizó de manera correcta, manteniendo el equilibrio original entre clases gracias al uso del parámetro stratify=y.

# Estadatizar X para la regresión logísitica
En las siguientes líneas de código vamos a aplicar el **escalamiento solo a las variables X que usaremos en la Regresión Logística**, dejando intactas las versiones originales para Random Forest. Para ello, el `StandardScaler` primero “aprende” la media y la desviación estándar de cada columna usando únicamente `X_train`, y luego utiliza esos mismos valores para transformar tanto `X_train` como `X_test`. De esta forma evitamos filtrar información del conjunto de prueba durante el entrenamiento y, al final, tendremos dos conjuntos de features: `X_train_log` y `X_test_log` (escalados para la Regresión Logística) y `X_train`, `X_test` sin escalar para Random Forest.


In [6]:
from sklearn.preprocessing import StandardScaler

# Crear el objeto escalador
scaler = StandardScaler()

# Ajustar el escalador SOLO con los datos de entrenamiento
scaler.fit(X_train)

# Transformar train y test con ese mismo escalador
X_train_log = scaler.transform(X_train)
X_test_log  = scaler.transform(X_test)

# Verificar dimensiones
X_train_log.shape, X_test_log.shape

((455, 30), (114, 30))

ahora, vamos a guardar el `StandardScaler` que acabamos de entrenar. Esto es importante porque, cuando el modelo se utilice en fases posteriores o en una aplicación final, necesitaremos aplicar exactamente la misma escala a cualquier dato nuevo. Guardar el objeto `scaler` en un archivo `.pkl` permite reutilizarlo sin tener que entrenarlo nuevamente, siguiendo el flujo real de trabajo en Machine Learning: preprocesar → entrenar → guardar → usar para predecir. Para ello, exportaremos el `scaler` dentro de una carpeta llamada **models/** y así podrá ser cargado fácilmente en la Fase 3.

In [ ]:
import joblib
import os

# Guardar el scaler entrenado
joblib.dump(scaler, "../models/scaler_logreg.pkl")

print("Scaler guardado exitosamente.")

Scaler guardado exitosamente.


# Organizar las variables que se usaran en el modelo
A continuación, vamos a dejar organizado y totalmente explícito qué conjuntos de datos usará cada modelo. Este pequeño bloque sirve para mantener una estructura clara antes de pasar a la Fase 3, evitando confusiones y asegurando que el flujo del proyecto se mantenga limpio y profesional. Además, es la transición natural dentro del pipeline: después de preprocesar los datos, dejamos definidos los insumos exactos con los que entrenaremos tanto la Regresión Logística como el Random Forest.

In [8]:
# ----------------------------
# Conjuntos finales para Fase 3
# ----------------------------

# Para Regresión Logística (requiere datos escalados)
X_train_logreg = X_train_log
X_test_logreg  = X_test_log

# Para Random Forest (NO requiere escalamiento)
X_train_rf = X_train
X_test_rf  = X_test

# Las etiquetas son las mismas para ambos modelos
y_train_final = y_train
y_test_final  = y_test

print("Conjuntos finales listos para entrenamiento.")


Conjuntos finales listos para entrenamiento.


In [9]:

# Crear carpeta para datos procesados (si no existe)
os.makedirs("data/processed", exist_ok=True)

# Guardar conjuntos para Regresión Logística
joblib.dump(X_train_logreg, "data/processed/X_train_logreg.pkl")
joblib.dump(X_test_logreg, "data/processed/X_test_logreg.pkl")

# Guardar conjuntos para Random Forest
joblib.dump(X_train_rf, "data/processed/X_train_rf.pkl")
joblib.dump(X_test_rf, "data/processed/X_test_rf.pkl")

# Guardar variable objetivo
joblib.dump(y_train_final, "data/processed/y_train_final.pkl")
joblib.dump(y_test_final, "data/processed/y_test_final.pkl")

print("✅ Conjuntos de entrenamiento y prueba guardados en data/processed/")


✅ Conjuntos de entrenamiento y prueba guardados en data/processed/
